In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import datetime as dt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [30]:
def extractTable(df, filename, sheetname, atRows, columns=None, year=None, month=None):
    df = pd.DataFrame()
    df = pd.read_excel(filename, sheet_name=sheetname, skiprows=atRows, usecols=columns)

    if(not(year) or not(month)):
        return df

    df["DATE"] = pd.Timestamp(year, month, 1)

    return df


In [31]:
def dataProcessing(df, columnsToFill, fillContent):
    for name in columnsToFill:
        df[name] = df[name].fillna(fillContent)
    return df

def replaceRowsValue(df, beforeList, afterList):
    for before, after in zip(beforeList, afterList):
        df = df.replace(before, after)
    return df

def renameColumns(df, columnsList):
    df = df.rename(columns=columnsList)
    return df

In [32]:
def tablebySheetMonths(df, filename, year, atRows, monthsList, columns=None):
    for idx, m in enumerate(monthsList):
        month = idx+1
        dftemp = pd.DataFrame()
        dftemp = extractTable(dftemp, filename, m, atRows, columns, year, month)

        df = pd.concat([df,dftemp])
    return df

In [33]:
def tablebySheetYears(df, filename, m, atRows, columns, yearsList):
    for idx, y in enumerate(yearsList):

        dftemp = pd.DataFrame()
        dftemp = extractTable(dftemp, filename, y, atRows, columns)

        df = pd.concat([df,dftemp])
    return df

In [34]:
def fixCustomersName(df, idx):
    for index in idx:
        dfPrevRow = df.iloc[[index-1]]
        dftemp = df.iloc[[index]]

        if(dftemp.SO.to_numpy() > 0):
            df.at[index,"CUSTOMER"] = dfPrevRow.CUSTOMER.to_numpy()[0]
        else:
            df = df.drop(index=index, axis=0)
    return df

In [35]:
rekapan_iso_2018 = pd.DataFrame()
rekapan_iso_2019 = pd.DataFrame()
rekapan_iso_2020 = pd.DataFrame()
rekapan_iso_2021 = pd.DataFrame()
rekapan_iso_2022 = pd.DataFrame()

rekapan_iso_2018 = extractTable(rekapan_iso_2018, "Data Penjualan All (2018-2022).xlsx", "2018", 0)
rekapan_iso_2019 = extractTable(rekapan_iso_2019, "Data Penjualan All (2018-2022).xlsx", "2019", 0)
rekapan_iso_2020 = extractTable(rekapan_iso_2020, "Data Penjualan All (2018-2022).xlsx", "2020", 0)
rekapan_iso_2021 = extractTable(rekapan_iso_2021, "Data Penjualan All (2018-2022).xlsx", "2021", 0)
rekapan_iso_2022 = extractTable(rekapan_iso_2022, "Data Penjualan All (2018-2022).xlsx", "2022", 0)

rekapan_iso_2022 = rekapan_iso_2022.iloc[:,:8]

In [36]:
rekapan_iso_2023_months = ["Januari", "Februari", "Maret", "April", "Mei", "Juni", "Juli", "Agustus", "September", "Oktober", "November", "Desember"]
rekapan_iso_2023_columns = [0, 1, 4, 5, 6, 8, 10]
rekapan_iso_2023 = pd.DataFrame()
rekapan_iso_2023 = tablebySheetMonths(rekapan_iso_2023, "REKAPAN_ISO_2023.xlsx", 2023, 9, rekapan_iso_2023_months, rekapan_iso_2023_columns)

In [37]:
rekapan_iso_2024_months = ["Januari", "Februari", "Maret", "April"]
rekapan_iso_2024_columns = [0, 1, 4, 5, 6, 8, 10]
rekapan_iso_2024 = pd.DataFrame()
rekapan_iso_2024 = tablebySheetMonths(rekapan_iso_2024, "REKAPAN_ISO_2023.xlsx", 2024, 9, rekapan_iso_2024_months, rekapan_iso_2024_columns)

In [38]:
rename_columns_List_2018_2019 = {
    "Tanggal":"TANGGAL",
    "Customer":"CUSTOMER",
    "Nama Barang":"NAMA PRODUK",
    "Terkirim":"TERKIRIM",
    "Sisa":"SISA PO",
    "Internal":"INTERNAL",
    "Eksternal":"EKSTERNAL"
}
rename_columns_List_2020 = {
    "Tanggal":"TANGGAL",
    "Customer":"CUSTOMER",
    "Nama Barang":"NAMA PRODUK",
    "Sisa":"SISA PO"
}
rename_columns_List_2021 = {
    "Tanggal":"TANGGAL",
    "Customer":"CUSTOMER",
    "Nama Barang":"NAMA PRODUK",
    "SISA":"SISA PO"
}
rename_columns_List_2022 = {
    "Tanggal":"TANGGAL",
    "PO":"SO"
}
rename_columns_List_2023_2024 = {
    "DATE":"TANGGAL",
    "PO.1":"SO"
}

In [39]:
rekapan_iso_2018 = renameColumns(rekapan_iso_2018, rename_columns_List_2018_2019)
rekapan_iso_2019 = renameColumns(rekapan_iso_2019, rename_columns_List_2018_2019)
rekapan_iso_2020 = renameColumns(rekapan_iso_2020, rename_columns_List_2020)
rekapan_iso_2021 = renameColumns(rekapan_iso_2021, rename_columns_List_2021)
rekapan_iso_2022 = renameColumns(rekapan_iso_2022, rename_columns_List_2022)
rekapan_iso_2023 = renameColumns(rekapan_iso_2023, rename_columns_List_2023_2024)

rekapan_iso_2024 = renameColumns(rekapan_iso_2024, rename_columns_List_2023_2024)

In [40]:
columns_to_fill_list = ["INTERNAL", "EKSTERNAL"]
rekapan_iso_2018 = dataProcessing(rekapan_iso_2018,columns_to_fill_list, " ")
rekapan_iso_2019 = dataProcessing(rekapan_iso_2019,columns_to_fill_list, " ")
rekapan_iso_2020 = dataProcessing(rekapan_iso_2020,columns_to_fill_list, " ")
rekapan_iso_2021 = dataProcessing(rekapan_iso_2021,columns_to_fill_list, " ")
rekapan_iso_2022 = dataProcessing(rekapan_iso_2022,columns_to_fill_list, " ")
rekapan_iso_2023 = dataProcessing(rekapan_iso_2023,columns_to_fill_list, " ")

rekapan_iso_2024 = dataProcessing(rekapan_iso_2024,columns_to_fill_list, " ")


# ----------------------------------------------------------------
rekapan_iso_2018 = dataProcessing(rekapan_iso_2018,['SO'], 0)
rekapan_iso_2019 = dataProcessing(rekapan_iso_2019,['SO'], 0)
rekapan_iso_2020 = dataProcessing(rekapan_iso_2020,['SO'], 0)
rekapan_iso_2021 = dataProcessing(rekapan_iso_2021,['SO'], 0)
rekapan_iso_2022 = dataProcessing(rekapan_iso_2022,['SO'], 0)
rekapan_iso_2023 = dataProcessing(rekapan_iso_2023,['SO'], 0)
rekapan_iso_2023 = dataProcessing(rekapan_iso_2023,['CUSTOMER'], "")

rekapan_iso_2024 = dataProcessing(rekapan_iso_2024,['SO'], 0)
rekapan_iso_2024 = dataProcessing(rekapan_iso_2024,['CUSTOMER'], "")

In [41]:
print(rekapan_iso_2018.shape)
print(rekapan_iso_2019.shape)
print(rekapan_iso_2020.shape)
print(rekapan_iso_2021.shape)
print(rekapan_iso_2022.shape)
print(rekapan_iso_2023.shape)
print(rekapan_iso_2024.shape)

(3133, 8)
(3041, 8)
(2359, 8)
(3129, 8)
(3642, 8)
(12494, 8)
(4204, 8)


In [42]:
rekapan_iso_2018 = rekapan_iso_2018[rekapan_iso_2018.SO > 0]
rekapan_iso_2019 = rekapan_iso_2019[rekapan_iso_2019.SO > 0]
rekapan_iso_2020 = rekapan_iso_2020[rekapan_iso_2020.SO > 0]
rekapan_iso_2021 = rekapan_iso_2021[rekapan_iso_2021.SO > 0]
rekapan_iso_2022 = rekapan_iso_2022[rekapan_iso_2022.SO > 0]
rekapan_iso_2023 = rekapan_iso_2023[rekapan_iso_2023.SO > 0]
rekapan_iso_2024 = rekapan_iso_2024[rekapan_iso_2024.SO > 0]

In [43]:
rekapan_iso_2023_idx = rekapan_iso_2023[rekapan_iso_2023["CUSTOMER"] == ""].index
rekapan_iso_2023 = fixCustomersName(rekapan_iso_2023, rekapan_iso_2023_idx)


rekapan_iso_2024_idx = rekapan_iso_2024[rekapan_iso_2024["CUSTOMER"] == ""].index
rekapan_iso_2024 = fixCustomersName(rekapan_iso_2024, rekapan_iso_2024_idx)

In [44]:
print(rekapan_iso_2018.shape)
print(rekapan_iso_2019.shape)
print(rekapan_iso_2020.shape)
print(rekapan_iso_2021.shape)
print(rekapan_iso_2022.shape)
print(rekapan_iso_2023.shape)
print(rekapan_iso_2024.shape)

(3133, 8)
(3041, 8)
(2359, 8)
(3129, 8)
(3303, 8)
(3532, 8)
(1184, 8)


In [45]:
rekapan_iso_2018['TANGGAL'] = pd.to_datetime(rekapan_iso_2018['TANGGAL']).dt.date
rekapan_iso_2019['TANGGAL'] = pd.to_datetime(rekapan_iso_2019['TANGGAL']).dt.date
rekapan_iso_2020['TANGGAL'] = pd.to_datetime(rekapan_iso_2020['TANGGAL']).dt.date
rekapan_iso_2021['TANGGAL'] = pd.to_datetime(rekapan_iso_2021['TANGGAL']).dt.date
rekapan_iso_2022['TANGGAL'] = pd.to_datetime(rekapan_iso_2022['TANGGAL']).dt.date
rekapan_iso_2023['TANGGAL'] = pd.to_datetime(rekapan_iso_2023['TANGGAL']).dt.date


rekapan_iso_2024['TANGGAL'] = pd.to_datetime(rekapan_iso_2024['TANGGAL']).dt.date

In [46]:
rekapan_iso_2018.set_index('TANGGAL', inplace=True)
rekapan_iso_2019.set_index('TANGGAL', inplace=True)
rekapan_iso_2020.set_index('TANGGAL', inplace=True)
rekapan_iso_2021.set_index('TANGGAL', inplace=True)
rekapan_iso_2022.set_index('TANGGAL', inplace=True)
rekapan_iso_2023.set_index('TANGGAL', inplace=True)
rekapan_iso_2024.set_index('TANGGAL', inplace=True)

In [47]:
customers_to_fix_list = [
    # ACC (X)
    # AISIN (1)
        "AISIN INDONESIA",
        "A"

    # AMESU (X)
    # AMINDO (1)
        "AMINDO JAYA",

    # ANEKA KOMKAR (1)
        "ANEKA KOMKAR ",

    # ANEKA RUBBERINDO PERANTI SEJATI (2)
        "ANEKA RUBERINDO",
        "ANEKA RUBERRINDO",

    # APM LEAF SPRING (4)
        "APM LEAF SPRING ",
        "APM LEAF SPRINGS",
        "APM",
        "APM LEAF SPRING INA",

    # ASKI (X)
    # ARINDA JAYA ABADI (X)
    # ASTRA OTOPARTS - NUSAMETAL (2)
        "NUSAMETAL",
        "NUSA METAL",

    # ASTRA OTOPARTS - ADIWIRA PLASTIK (3)
        "ASTRA OTOPARTS",
        "ASTRA OTOPART",
        "AWP",

    # AUTOPLASTIK (X)

    #Total A: 14
    ################################################
    # BETA MEGI UTAMA (1)
        "BETAMEGI UTAMI",

    # BONECOM TRICOM (1)
        "BONECOM",


    #Total B: 2
    ################################################
    # CHANDRA NUGERAH CIPTA (1)
        "CHANDRA NUGERAH C",

    # CHEMCO HARAPAN NUSANTARA (1)
        "CHEMCO",

    # CHUHATSU (X)
    # CITRA GALVALINDO SUKSEK MANDIRI (1)
        "CITRA GALVALINDO",

    # CITRA LANGGENG SENTOSA (4)
        "SENTOSA",
        "CITRA LANGGENG",
        "CITRA LANGGENG ",
        "CITRA LANGGENG S",

    # CKPM (X)
    # CPM (X)


    #Total C: 7
    ################################################
    # DARWIN SETIAWAN (X)
    # DETRACON (X)
    # DUTA NICHRINDO PRATAMA (2)
        "DNP",
        "DUTA NICHIRINDO",

    # DWI ALAM LESTARI (X)

    # DWI UTAMA (X)

    #Total D: 2
    ################################################
    # EKAMITRA JAYA (X)
    # ERAN PLASTINDO UTAMA (2)
        "ERAN PLASTINDO",
        "ERAN PLASTINDO U",

    #Total E: 2
    ################################################
    # GARUDA METAL UTAMA (2)
        "GMU",
        "GARUDA METAL UTAMA ",

    # GARUDA METALINDO (1)
        "GARUDA METALINDO ",

    # GEMA GRAHA SARANA (3)
        "GEMA GRAHA",
        "GEMA GRAHA ",
        "GEMA GRAHA S",

    # GINSA INTI PRATAMA (1)
        "GINSA",

    # GLOBAL DIMENSI METALINDO (X)
    # GREAT GIANT PINEAPPLE (1)
        "GREAT GIANT PINE",

    # GUNA SENAPUTRA SEJAHTERA (2)
        "GUNA SENA",
        "GUNA SENAPUTRA",
        "GUNASENA PUTRA",

    #Total G: 11
    ################################################
    # HANDAL CITRA (X)
    # HI-LEX (7)
        "PT HILEX",
        "PT HILEX ",
        "HILEX",
        "HILEX ",
        "HILEX INDONESIA",
        "HI-LEX INDOENSIA",
        "HI-LEX INDONESIA",

    # HINO MOTOR SALES (1)
        "HINO",

    #Total H: 8
    ################################################
    # INDOKARLO PERKASA (X)
    # INDOSEIKI METAL UTAMA (X)
    # INDRA GRAHA (1)
        "INDRAGRAHA",

    # INDTA PRAMAJAYA (X)
    # INJEKSI PLASTIK PASIFIK (X)
    # INTERKARTALINDO (X)
    # ISUZU ASTRA MOTOR (1)
        "ISUZU",
        "ISUZU ASTRA",


    #Total I: 9
    ################################################
    # JARISKO JAYA CIPTA (2)
        "JARISKO",
        "JARISKO JAYA",
    # JAYA MANDIRI MOULD (X)

    #Total J: 2
    ################################################
    # KAYABA (1)
        "PT KAYABA",

    # KENARI DJAJA PRIMA (2)
        "KENARI",
        "KENARI DJAYA",

    # KONGO CITRA MANUFAKTUR (X)


    #Total K: 3
    ################################################
    # LESTARI TEHNIK PLASTIKATAMA(2)
        "LESTARI TEHNIK",
        "LESTARI TEKNIK",

    #Total L: 2
    ################################################
    # MARUTAKE MIYAMA (2)
        "MARUTAKE",
        "MARUTAKE ",

    # MENARA TERUS MAKMUR (1)
        "MTM",

    # METAL CORFIX (2)
        "METAL CORFIK",
        "METAL CORFIX INDONESIA",

    # METAL FASTINDO (1)
        "METAL FASTINDO ",
    # METALINDO (X)
    # MITRA SANJAYA (X)
    # MITRAMETAL PERKASA (2)
        "MITRA METAL",
        "MITRAMETAL",

    # MOLDEN ANUGERAH SEJAHTERA (1)
        "MOLDEN ANUGERAH SEJAHTERA ",

    # MOLDEN PATRA SEJATA (X)
    # MUARATEWEH SPRING (2)
        "MUARA TEWEH",
        'MUARATEWEH',


    #Total M: 11
    ################################################
    # NAKAKIN (X)
    # NITTO ALAM (1)
        "NITTO ALAM INDONESIA",


    #Total N: 1
    ################################################
    # PBR (X)
    # PLATINUM BERKAT TEKNINDO (2)
        "PLATINUM BERKAT",
        "PLATINUM BERKAT T",

    # PREFORMED LINE PRODUCTS (1)
        "PLP",

    # PRIMA REJEKI (X)
    # PRIUK PERKASA (2)
        "PERIUK PERKASA",
        "PERIUK PERKASA ",

    #Total P: 5
    ################################################
    # REKA GRAHA SEMESTA (2)
        "REKA GRAHA",
        "REKA GRAHA ",

    # RIMBA MUTIARA KUSUMA (X)

    #Total R: 2
    ################################################
    # SANWA ENGINERING  (3)
        "SANWA ",
        "SANWA ENGINERING INDON",
        "SANWA ENGINERING INDONESIA",

    # SETIA GUNA SEJATI (1)
        "SETIAGUNA SEJATI",

    # SETIA GUNA SELARAS (X)

    # SGS (X)
    # SINAR ALUM SARANA (1)
        "SINAR ALUM",

    # STAR MUSTIKA PLASTMETAL (2)
        "STAR MUSTIKA",
        "STAR MUSTIKA ",

    # SUMARA (X)
    # SURYA JAYA TEKNIK (X)


    #Total S: 7
    ################################################
    # TBINA (X)
    # TIGA KARUNIA (3)
        "TIGA KARUNIA ",
        "TIGA KARUNIA INDO",
        "TIGA KARUNIA INDONESIA",

    # TOA GALINDRA ELECTRONICS (2)
        "TOA GALINDRA",
        "TOA GALINDRA ",

    # TOA GALVA INDUSTRIES (4)
        "TOA",
        "TOA GALVA",
        "PT TOA GALVA",
        "TOA GALVA CIKARANG",

    # TOYOTA BOSHOKU (X)
    # TRICIPTA TEKNINDO (X)
        "TRICIPTA TEJNINDO",
        "TRI CIPTA TEKNINDO",

    # TSURUTA (2)
        "TSURUTA ",
        "TSURUTA INDOENSIA",


    #Total T: 13
    ################################################
    # VELASTO (2)
        "VELASTO INDONESIA",
        "VELASTO INDONESIA PT",


    #Total V: 2
    ################################################
    # WIJAYA MAJU (X)

    #Total W: 0
    ################################################

]

In [48]:
correct_costumers_list = [
    # ACC (X)
    # AISIN (1)
        "AISIN",

    # AMESU (X)
    # AMINDO (1)
        "AMINDO",

    # ANEKA KOMKAR (1)
        "ANEKA KOMKAR",

    # ANEKA RUBBERINDO PERANTI SEJATI (2)
        "ANEKA RUBBERINDO PERANTI SEJATI",
        "ANEKA RUBBERINDO PERANTI SEJATI",

    # APM LEAF SPRING (4)
        "APM LEAF SPRING",
        "APM LEAF SPRING",
        "APM LEAF SPRING",
        "APM LEAF SPRING",

    # ASKI (X)
    # ARINDA JAYA ABADI (X)
    # ASTRA OTOPARTS - NUSAMETAL (2)
        "ASTRA OTOPARTS - NUSAMETAL",
        "ASTRA OTOPARTS - NUSAMETAL",

    # ASTRA OTOPARTS - ADIWIRA PLASTIK (3)
        "ASTRA OTOPARTS - ADIWIRA PLASTIK",
        "ASTRA OTOPARTS - ADIWIRA PLASTIK",
        "ASTRA OTOPARTS - ADIWIRA PLASTIK",

    # AUTOPLASTIK (X)

    #Total A: 14
    ################################################
    # BETA MEGI UTAMA (1)
        "BETA MEGI UTAMA",

    # BONECOM TRICOM (1)
        "BONECOM TRICOM",


    #Total B: 2
    ################################################
    # CHANDRA NUGERAH CIPTA (1)
        "CHANDRA NUGERAH CIPTA",

    # CHEMCO HARAPAN NUSANTARA (1)
        "CHEMCO HARAPAN NUSANTARA",

    # CHUHATSU (X)
    # CITRA GALVALINDO SUKSEK MANDIRI (1)
        "CITRA GALVALINDO SUKSEK MANDIRI",

    # CITRA LANGGENG SENTOSA (4)
        "CITRA LANGGENG SENTOSA",
        "CITRA LANGGENG SENTOSA",
        "CITRA LANGGENG SENTOSA",
        "CITRA LANGGENG SENTOSA",

    # CKPM (X)
    # CPM (X)

    #Total C: 7
    ################################################
    # DARWIN SETIAWAN (X)
    # DETRACON (X)
    # DUTA NICHRINDO PRATAMA (2)
        "DUTA NICHRINDO PRATAMA",
        "DUTA NICHRINDO PRATAMA",

    # DWI ALAM LESTARI (X)

    # DWI UTAMA (X)

    #Total D: 2
    ################################################
    # EKAMITRA JAYA (X)
    # ERAN PLASTINDO UTAMA (2)
        "ERAN PLASTINDO UTAMA",
        "ERAN PLASTINDO UTAMA",

    #Total E: 2
    ################################################
    # GARUDA METAL UTAMA (2)
        "GARUDA METAL UTAMA",
        "GARUDA METAL UTAMA",

    # GARUDA METALINDO (1)
        "GARUDA METALINDO",

    # GEMA GRAHA SARANA (3)
        "GEMA GRAHA SARANA",
        "GEMA GRAHA SARANA",
        "GEMA GRAHA SARANA",

    # GINSA INTI PRATAMA (1)
        "GINSA INTI PRATAMA",

    # GLOBAL DIMENSI METALINDO (X)
    # GREAT GIANT PINEAPPLE (1)
        "GREAT GIANT PINEAPPLE",

    # GUNA SENAPUTRA SEJAHTERA (3)
        "GUNA SENAPUTRA SEJAHTERA",
        "GUNA SENAPUTRA SEJAHTERA",
        "GUNA SENAPUTRA SEJAHTERA",

    #Total G: 11
    ################################################
    # HANDAL CITRA (X)
    # HI-LEX (7)
        "HI-LEX",
        "HI-LEX",
        "HI-LEX",
        "HI-LEX",
        "HI-LEX",
        "HI-LEX",
        "HI-LEX",

    # HINO MOTOR SALES (1)
        "HINO MOTOR SALES",

    #Total H: 8
    ################################################
    # INDOKARLO PERKASA (X)
    # INDOSEIKI METAL UTAMA (X)
    # INDRA GRAHA (1)
        "INDRA GRAHA",

    # INDTA PRAMAJAYA (X)
    # INJEKSI PLASTIK PASIFIK (X)
    # INTERKARTALINDO (X)
    # ISUZU ASTRA MOTOR (2)
        "ISUZU ASTRA MOTOR",
        "ISUZU ASTRA MOTOR",



    #Total I: 9
    ################################################
    # JARISKO JAYA CIPTA (2)
        "JARISKO JAYA CIPTA",
        "JARISKO JAYA CIPTA",

    # JAYA MANDIRI MOULD (X)

    #Total J: 2
    ################################################
    # KAYABA (1)
        "KAYABA",

    # KENARI DJAJA PRIMA (2)
        "KENARI DJAJA PRIMA",
        "KENARI DJAJA PRIMA",

    # KONGO CITRA MANUFAKTUR (X)


    #Total K: 3
    ################################################
    # LESTARI TEHNIK PLASTIKATAMA(2)
        "LESTARI TEHNIK PLASTIKATAMA",
        "LESTARI TEHNIK PLASTIKATAMA",

    #Total L: 2
    ################################################
    # MARUTAKE MIYAMA (2)
        "MARUTAKE MIYAMA",
        "MARUTAKE MIYAMA",

    # MENARA TERUS MAKMUR (1)
        "MENARA TERUS MAKMUR",

    # METAL CORFIX (2)
        "METAL CORFIX",
        "METAL CORFIX",

    # METAL FASTINDO (1)
        "METAL FASTINDO",

    # METALINDO (X)
    # MITRA SANJAYA (X)
    # MITRAMETAL PERKASA (2)
        "MITRAMETAL PERKASA",
        "MITRAMETAL PERKASA",

    # MOLDEN ANUGERAH SEJAHTERA (1)
        "MOLDEN ANUGERAH SEJAHTERA",

    # MOLDEN PATRA SEJATA (X)
    # MUARATEWEH SPRING (2)
        "MUARATEWEH SPRING",
        'MUARATEWEH SPRING' ,


    #Total M: 11
    ################################################
    # NAKAKIN (X)
    # NITTO ALAM (1)
        "NITTO ALAM",


    #Total N: 1
    ################################################
    # PBR (X)
    # PLATINUM BERKAT TEKNINDO (2)
        "PLATINUM BERKAT TEKNINDO",
        "PLATINUM BERKAT TEKNINDO",

    # PREFORMED LINE PRODUCTS (1)
        "PREFORMED LINE PRODUCTS",

    # PRIMA REJEKI (X)
    # PRIUK PERKASA (1)
        "PRIUK PERKASA",
        "PRIUK PERKASA",

    #Total P: 5
    ################################################
    # REKA GRAHA SEMESTA (2)
        "REKA GRAHA SEMESTA",
        "REKA GRAHA SEMESTA",

    # RIMBA MUTIARA KUSUMA (X)

    #Total R: 2
    ################################################
    # SANWA ENGINERING  (3)
        "SANWA ENGINERING",
        "SANWA ENGINERING",
        "SANWA ENGINERING",

    # SETIA GUNA SEJATI (1)
        "SETIAGUNA SEJATI",

    # SETIA GUNA SELARAS (X)
    # SGS (X)
    # SINAR ALUM SARANA (1)
        "SINAR ALUM SARANA",

    # STAR MUSTIKA PLASTMETAL (2)
        "STAR MUSTIKA PLASTMETAL",
        "STAR MUSTIKA PLASTMETAL",

    # SUMARA (X)
    # SURYA JAYA TEKNIK (X)


    #Total S: 7
    ################################################
    # TBINA (X)
    # TIGA KARUNIA (2)
        "TIGA KARUNIA",
        "TIGA KARUNIA",
        "TIGA KARUNIA",

    # TOA GALINDRA ELECTRONICS (1)
        "TOA GALINDRA ELECTRONICS",
        "TOA GALINDRA ELECTRONICS",

    # TOA GALVA INDUSTRIES (4)
        "TOA GALVA INDUSTRIES",
        "TOA GALVA INDUSTRIES",
        "TOA GALVA INDUSTRIES",
        "TOA GALVA INDUSTRIES",

    # TOYOTA BOSHOKU (X)
    # TRICIPTA TEKNINDO (2)
        "TRICIPTA TEKNINDO",
        "TRICIPTA TEKNINDO",

    # TSURUTA (1)
        "TSURUTA",
        "TSURUTA",


    #Total T: 13
    ################################################
    # VELASTO (2)
        "VELASTO",
        "VELASTO",


    #Total V: 1
    ################################################
    # WIJAYA MAJU (X)

    #Total W: 0
    ################################################

]

In [49]:
print("to fix list",len(customers_to_fix_list))
print("correct list",len(correct_costumers_list))


to fix list 97
correct list 97


In [50]:
rekapan_iso_2018 = replaceRowsValue(rekapan_iso_2018, customers_to_fix_list, correct_costumers_list)
rekapan_iso_2019 = replaceRowsValue(rekapan_iso_2019, customers_to_fix_list, correct_costumers_list)
rekapan_iso_2020 = replaceRowsValue(rekapan_iso_2020, customers_to_fix_list, correct_costumers_list)
rekapan_iso_2021 = replaceRowsValue(rekapan_iso_2021, customers_to_fix_list, correct_costumers_list)
rekapan_iso_2022 = replaceRowsValue(rekapan_iso_2022, customers_to_fix_list, correct_costumers_list)
rekapan_iso_2023 = replaceRowsValue(rekapan_iso_2023, customers_to_fix_list, correct_costumers_list)

rekapan_iso_2024 = replaceRowsValue(rekapan_iso_2024, customers_to_fix_list, correct_costumers_list)

In [51]:
rekapan_iso_all = pd.concat(
    [
        rekapan_iso_2018,
        rekapan_iso_2019,
        rekapan_iso_2020,
        rekapan_iso_2021,
        rekapan_iso_2022,
        rekapan_iso_2023,
        rekapan_iso_2024,
    ]
)

In [52]:
with pd.ExcelWriter("rekapan_iso_all_year_clean.xlsx") as writer:
    rekapan_iso_2018.to_excel(writer, sheet_name="2018", index=True)
    rekapan_iso_2019.to_excel(writer, sheet_name="2019", index=True)
    rekapan_iso_2020.to_excel(writer, sheet_name="2020", index=True)
    rekapan_iso_2021.to_excel(writer, sheet_name="2021", index=True)
    rekapan_iso_2022.to_excel(writer, sheet_name="2022", index=True)
    rekapan_iso_2023.to_excel(writer, sheet_name="2023", index=True)
    rekapan_iso_2024.to_excel(writer, sheet_name="2024", index=True)
    rekapan_iso_all.to_excel(writer, sheet_name="rekapan", index=True)

In [53]:
rekapan_iso_2018_to_2023 = pd.concat(
    [
        rekapan_iso_2018,
        rekapan_iso_2019,
        rekapan_iso_2020,
        rekapan_iso_2021,
        rekapan_iso_2022,
        rekapan_iso_2023,
    ]
)
showCustomers = rekapan_iso_2018_to_2023['CUSTOMER'].sort_values().drop_duplicates()
print(showCustomers.to_string())

TANGGAL
2018-11-01                                 ACC
2023-03-01                               AISIN
2018-08-01                               AMESU
2022-07-01                              AMINDO
2023-01-01                         AMINDO JAYA
2018-08-01              AMINDO JAYA DIESELTAMA
2019-01-01                        ANEKA KOMKAR
2022-01-01     ANEKA RUBBERINDO PERANTI SEJATI
2023-09-01                     APM LEAF SPRING
2023-06-01                   ARINDA JAYA ABADI
2020-03-01                                ASKI
2020-01-01                      ASTRA KOMPONEN
2019-08-01    ASTRA OTOPARTS - ADIWIRA PLASTIK
2019-11-01          ASTRA OTOPARTS - NUSAMETAL
2018-08-01                         AUTOPLASTIK
2021-09-01                        BANGUN ERA S
2018-08-01                           BERDIKARI
2023-05-01                     BETA MEGI UTAMA
2022-10-01                     BOHLINDO TEHNIK
2023-03-01                      BONECOM TRICOM
2018-11-01                        CAHAYA BUKIT
2022-

In [54]:
# print(rekapan_iso_2018)
# print(rekapan_iso_2019)
# print(rekapan_iso_2020)
# print(rekapan_iso_2021)
# print(rekapan_iso_2022)
# print(rekapan_iso_2023)